<a href="https://colab.research.google.com/github/Prajaktahz/ML-Practice-Uni/blob/main/Practical_7_(sklearn_version)_Variable_Importance_%26_Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://www.cs.nott.ac.uk/~pszgss/teaching/nlab.png)
<h1 style="text-align: center;">ML Practical: Variable Importance & Feature Selection</h1>

**At the end of this practical you should be able to:**
1. Compute variable importance measures using both *filter* and *permutation importance* measures and understand the difference
2. Perform and understand the difference between different feature selection methods (minimally optimal)

The data set for the practical is an **Online News Popularity Data Set** where the task is to predict how many shares an article will get as a binary value ("low" or "high").

## This practical is in two parts. The first part will be done as a demonstration. The second part is for you to do.



# Demo
## Demo Part 1: Creating a function to compare and print our variable importances
Since we want to compare different methods for computing variable importance we are going to want to print them side-by-side ordered by ranking.

While we could write the code each time, it gets a bit repeative. So we'll write a function that takes a dictionary of feature_importance scores and the list of feature names and prints the table.

*Why a dictionary?* As each list of feature importance scores needs a name, so key = name of method, value = feature importance scores.

**You do not have to understand this function for this practical** - just what it does and how to call it. I.e. read the documentation at the top of the function. That said, you should be able to understand it, it only contains python you know/know how to look up. If you've time once you've finished this practical look back of the function and try and understand it. Ask if you have any questions.

In [ ]:
def print_variable_importances( feature_names, dict_in, show_top = 10 ):
  """
  Prints a table of feature importance scores.

  Keyword arguments
  feature_names -- list of feature names. Must have the same ordering as the
                   scores in each instance of list_of_scores (see below)
  dict_in       -- dictionary of the form {method_name:list_of_scores} where:
                   method_name    -- string
                   list_of_scores -- list of scores, ordered in the same order
                                     as the passed feature_names
  show_top      -- number of features to show (default 10, None to print all)
  """


  # Implement the definition of None for show_top
  if show_top is None:
    show_top = len(feature_names)

  # Set up lists to hold the titles and score_feature tuples
  # We need a list so that they maintain fixed order as we
  # iterate over them row-by-row.
  to_print_titles = []
  to_print_scores = []

  # Pair each list of scores with a copy of the feature names and sort
  # based on the variable importance score descending
  for k, v in dict_in.items():
    # zip pairs, sorted sorts, reverse orders descending
    feature_names_plus_scores = sorted( zip(v, feature_names) )
    feature_names_plus_scores.reverse()
    to_print_titles.append(k)
    to_print_scores.append(feature_names_plus_scores)


  # Print the scores

  # Create a list of strings to print in each header cell
  line_parts = []
  for j in range(len(to_print_titles)):
    line_parts.append('{:<38}'.format(to_print_titles[j]))

  # Print each header cell using a separator ' | ', adding the fixed rank column header
  print('Rank | ' + ' | '.join( ['{:<38}'.format(x) for x in to_print_titles] ) )

  # Print the header underline
  print('---- + ' + ' + '.join( [ '-'*38 ]*len(to_print_titles) ) )

  # Print each line
  for i in range(show_top):
    # Create a list of strings to print for each row cell
    line_parts = []
    for j in range(len(to_print_titles)):
      line_parts.append(  '{:<30}: {:.4f}'.format(to_print_scores[j][i][1], to_print_scores[j][i][0]) )
    # Print the row by:
    # (1) joining each row cell using a separator ' | '
    # (2) adding the fixed rank column header
    print( '{:<4} | '.format(str(i)) + ' | '.join(line_parts) )




## Demo Part 2: Data loading and create our training and test splits
You should be familar with this by now. The data set we will use is has been uploaded to
<br>http://www.cs.nott.ac.uk/~pszgss/teaching/OnlineNewsPopularity.csv <br>
Full details about the dataset can be found at:
<br>https://archive.ics.uci.edu/ml/datasets/online+news+popularity


In [ ]:
# Load the data
# Check the column names (i.e. print them). What is wrong? Yes, you will need to fix that.
# HINT: Strings have a .strip() method which returns the string without any leading or training whitespace.
#       I'd advise creating a new (corrected) array of column headings and update data.columns


from pandas import read_csv
data = read_csv('http://www.cs.nott.ac.uk/~pszgss/teaching/OnlineNewsPopularity.csv')

# Fix the column headings
new_header = []
for x in data.columns:
  new_header.append( x.strip() )

data.columns = new_header

In [ ]:
data.columns

Index(['url', 'timedelta', 'n_tokens_title', 'n_tokens_content',
       'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens',
       'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos',
       'average_token_length', 'num_keywords', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
       'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg',
       'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares',
       'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday',
       'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
       'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00',
       'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',
     

In [ ]:
data.head(3)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.5,-0.1875,0.0,0.1875,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.0,0.0000,0.5,0.0000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.0,0.0000,0.5,0.0000,1500


We now need to select the input and output features.<br>
**HINT:** Not all input features should be included.

The output feature currently is continuous. Recall that, for this task, we want to predict how many shares an article will get as a binary value ("low" or "high").

* High is defined as shares >= 1400
* Low is defined as shares < 1400

In [ ]:
# Select the input and output features
# url and timedelta are considered non-predictive in the documentation and
# therefore should not be used as an input feature
X = data.drop(columns=['url','timedelta','shares'])
y = data.shares
y.values[y <  1400] = 0
y.values[y >=  1400] = 1

Finally we need to split our data into test and training sets.

We will perform all feature selection based on the training set ONLY.

The test set will be used to test if our feature selection has helped or hurt.

Use a `test_size=0.75` and a `random_state=42`.

Stratify your sample. There is no need to down or upsample as the sample is approximatly balanced.

Why 75% for the test set? As we want the code to execute quickly so you can get more done in the practical. In reality we would not set the test set this high. Once you have complete the practical feel free to change this and see the impact it has.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42, stratify = y)

## Demo Part 3: Removing features with low variance
The first thing to do is to check to see if any features (nearly) always have the same value.

This is done via a [`VarianceThreshold`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html) object. The `VarianceThreshold` Object takes one parameter:
* threshold -- the variance threshold below which to remove features

`VarianceThreshold` follows the **fit transform** interface and can be used as a pre-processing step within a pipeline.

**`.fit(...)`:** Learns which features have low variance.<br>
**`.transform(...)`:** Returns a copy of the data removing features with variance below the threshold.

After calling `.fit(..)` you can use the:<br>
`.get_support()` method to get a boolean mask of which features was selected.

Assuming your VarianceThreshold object is called vt, then you can see the list of features that were selected via:<br>
`X_train.columns[vt.get_support()]`
<br>If you do not understand what that line of code is doing, please ask.

**TASKS:**
1. Using the methods detailed above write the code to print the selected features with a threshold of zero.
2. The function `numpy.invert(...)` will invert a binary mask. Write the code to print the removed features rather than those kept.
3. Are any features removed?
4. What happens if you change the threshold?
5. What would a good value be?

In [ ]:
import numpy as np
from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold = 0.01)
vt.fit(X_train)
np.invert( vt.get_support() )

X_train.columns[np.invert(vt.get_support())]


Index(['global_sentiment_polarity', 'global_rate_positive_words',
       'global_rate_negative_words', 'min_positive_polarity',
       'max_negative_polarity'],
      dtype='object')

In [ ]:
X_train.columns[ np.invert(vt.get_support()) ]

Index(['global_sentiment_polarity', 'global_rate_positive_words',
       'global_rate_negative_words', 'min_positive_polarity',
       'max_negative_polarity'],
      dtype='object')

## Demo Part 4: Univariate variable importance  (known as filter based methods when used for selection)
Features can be ranked by (typically simple) measures of how they **individually** affect the output variable. There are many measures we can use, [a number of them are implemented in sklearn](http://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection).

Univariate feature ranking and selection (all measures) are all implemented in sklearn via the [`GenericUnivariateSelect`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.GenericUnivariateSelect.html) object.

`GenericUnivariateSelect` follows the **fit transform** interface and can be used as a pre-processing step within a pipeline.

**`GenericUnivariateSelect(...)`:** Constructor. Takes parameters which specify which univariate feature importance measure will be calculated (when calling `.fit(...)`) and the strategy to select the "best" features for creating a reduced feature dataset (when calling `.transform(...)`). See the documentation for the list of avaliable univariate scoring functions.

**`.fit(...)`:** Computes variable importance scores. After calling `.fit(...)` the ranking is avaliable within the object as the attribute `.scores_`<br>

**`.transform(...)`:** Takes the input features and creates a new input features dataset (transforms the original dataset into the new dataset) by selecting a subset of the features as defined by the paramters `mode` and `param`. For instance, if the following parameters were passed when creating the GenericUnivariateSelect object (via the cons)  `mode = 'k_best'` and `param = 10`, `.transform(...)` will return a dataset with only the top 10 input features as ranked.

If we only want to rank we still set up a selector and call `.fit(..)` to compute the rankings, but simply do not end up selecting features based on it (via `.transform(...)`).

After calling `.fit(...)` the scores are available as the attribute: `.scores_`

**TASK:**
1. First ranking the features via the scoring function: `mutual_info_classif`
2. Create a new dataset `X_train_b` by selecting only the top 10 features.
3. Unrelated to (1) & (2) create a Pipeline that incorporates kBest features selection via `to mutual_info_classif` where k = 10.


In [ ]:
from sklearn.feature_selection import GenericUnivariateSelect, mutual_info_classif

In [ ]:
# Demo Task 4.1: First create the GenericUnivariateSelect object, then fit it to compute the scores

selector = GenericUnivariateSelect( score_func = mutual_info_classif, mode = 'k_best', param = 10)

selector.fit(X_train, y_train)



GenericUnivariateSelect(mode='k_best', param=10,
                        score_func=<function mutual_info_classif at 0x7adc245fb640>)

In [ ]:
sorted(list(zip(selector.scores_,X_train.columns)))

[(0.0, 'abs_title_sentiment_polarity'),
 (0.0, 'average_token_length'),
 (0.0, 'avg_positive_polarity'),
 (0.0, 'data_channel_is_bus'),
 (0.0, 'data_channel_is_socmed'),
 (0.0, 'global_sentiment_polarity'),
 (0.0, 'kw_max_max'),
 (0.0, 'max_negative_polarity'),
 (0.0, 'min_positive_polarity'),
 (0.0, 'n_unique_tokens'),
 (0.0, 'num_self_hrefs'),
 (0.0, 'num_videos'),
 (0.0, 'weekday_is_friday'),
 (0.0, 'weekday_is_monday'),
 (0.0, 'weekday_is_thursday'),
 (0.0, 'weekday_is_tuesday'),
 (0.0, 'weekday_is_wednesday'),
 (1.3194129602434046e-05, 'num_keywords'),
 (0.0003113944678845293, 'title_sentiment_polarity'),
 (0.0005093956323696247, 'n_non_stop_words'),
 (0.0006496129606234913, 'n_non_stop_unique_tokens'),
 (0.0013245570835589415, 'avg_negative_polarity'),
 (0.0014783433263139134, 'data_channel_is_lifestyle'),
 (0.00173034695812313, 'global_rate_positive_words'),
 (0.001835848703227505, 'weekday_is_saturday'),
 (0.0023183466812881637, 'is_weekend'),
 (0.002969097120669284, 'n_tokens_

In [ ]:
# We will then add the features and their scores to a dictionary.
# We will do this for each scoring method so we can print the comparisons
# based on the function written at the start of this practical.
# The basic structure for adding to this dictionary and calling the function is
# bellow.
#
# TASK: Fill in the ? to add the computed variable importance scores.

feature_importance_scores = {}
feature_importance_scores['Filter'] = selector.scores_

print_variable_importances( X_train.columns, feature_importance_scores, show_top = 40 )

Rank | Filter                                
---- + --------------------------------------
0    | kw_max_avg                    : 0.0384
1    | self_reference_avg_sharess    : 0.0340
2    | LDA_03                        : 0.0296
3    | self_reference_min_shares     : 0.0276
4    | kw_avg_avg                    : 0.0251
5    | LDA_00                        : 0.0242
6    | LDA_04                        : 0.0240
7    | LDA_02                        : 0.0225
8    | self_reference_max_shares     : 0.0222
9    | kw_min_avg                    : 0.0215
10   | kw_max_min                    : 0.0213
11   | LDA_01                        : 0.0194
12   | kw_min_max                    : 0.0182
13   | data_channel_is_world         : 0.0171
14   | n_tokens_title                : 0.0125
15   | global_subjectivity           : 0.0100
16   | max_positive_polarity         : 0.0089
17   | num_hrefs                     : 0.0079
18   | min_negative_polarity         : 0.0075
19   | kw_avg_min                 

In [ ]:
# Demo Task 4.2: Create a new dataset X_train_b by selecting only the top 10 features.
X_train_b = selector.transform(X_train)


In [ ]:
X_train_b.shape

(9911, 10)

In [ ]:
# Demo Task 4.3: Unrelated to (1) & (2) create a Pipeline that incorporates kBest features selection via to mutual_info_classif where k = 10.
# Terminate the Pipeline with a KNeighborsClassifier
# Evaluate the performance of the classifier. Think about other pre-processing steps that might also be useful and include them.

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score



steps = [('select',StandardScaler()),
         ('ss', GenericUnivariateSelect( score_func = mutual_info_classif, mode = 'k_best', param = 10)),
         ('model', KNeighborsClassifier())]

knn = Pipeline( steps = steps )

knn.fit(X_train,y_train)

y_pred = knn.predict(X_test)

scores = accuracy_score(y_test, y_pred)

print(scores)



0.5916658258500656


In [ ]:
# Of course 10 may or may not be the best number to pick. If you feel like it look
# back at the scores and try another value.

steps = [('select',StandardScaler()),
         ('ss', GenericUnivariateSelect( score_func = mutual_info_classif, mode = 'k_best', param = 32)),
         ('model', KNeighborsClassifier())]

knn = Pipeline( steps = steps )

knn.fit(X_train,y_train)

y_pred = knn.predict(X_test)

scores = accuracy_score(y_test, y_pred)

print(scores)

0.6095247704570679


# End of Demo. Your turn.
## Part 1: Model Based Methods
Rather than using external measures of how features **individually** affect the output feature one can rank features by how much they are used in a trained model. An example of this is Random Forests.

In sklearn, Random Forests have an attribute `feature_importances_` which stores the scores each feature as a list denoting their importance. These scores are computed when the model is fit.

**TASK:**
1. Train a RandomForestClassifier and print out a list of feature importances

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf.fit(X_train, y_train)

rf.feature_importances_

array([0.0162212 , 0.02476212, 0.0253116 , 0.00011973, 0.02578386,
       0.02148413, 0.01326849, 0.01485121, 0.00816438, 0.02557019,
       0.01134817, 0.00172072, 0.00702524, 0.00225036, 0.00293432,
       0.00502811, 0.0052196 , 0.00651777, 0.02702512, 0.0298247 ,
       0.01613546, 0.00913756, 0.02871362, 0.02184573, 0.04165189,
       0.04278875, 0.03103284, 0.02534999, 0.03099166, 0.00298223,
       0.00325596, 0.00321724, 0.00304988, 0.003094  , 0.00426773,
       0.00265627, 0.0117312 , 0.02779228, 0.02960312, 0.03350304,
       0.0269951 , 0.03074201, 0.02741077, 0.02562807, 0.02576137,
       0.02276226, 0.02051887, 0.02091801, 0.02409789, 0.01262241,
       0.01032296, 0.02395554, 0.01459973, 0.01388344, 0.01282769,
       0.01511261, 0.01287966, 0.0117302 ])

In [ ]:
feature_importance_scores['RF'] = rf.feature_importances_

print_variable_importances( X_train.columns, feature_importance_scores, show_top = 10 )

Rank | Filter                                 | RF                                    
---- + -------------------------------------- + --------------------------------------
0    | kw_max_avg                    : 0.0384 | kw_avg_avg                    : 0.0428
1    | self_reference_avg_sharess    : 0.0340 | kw_max_avg                    : 0.0417
2    | LDA_03                        : 0.0296 | LDA_02                        : 0.0335
3    | self_reference_min_shares     : 0.0276 | self_reference_min_shares     : 0.0310
4    | kw_avg_avg                    : 0.0251 | self_reference_avg_sharess    : 0.0310
5    | LDA_00                        : 0.0242 | LDA_04                        : 0.0307
6    | LDA_04                        : 0.0240 | kw_avg_min                    : 0.0298
7    | LDA_02                        : 0.0225 | LDA_01                        : 0.0296
8    | self_reference_max_shares     : 0.0222 | kw_avg_max                    : 0.0287
9    | kw_min_avg                    : 0.02

## Part 2a: Permutation Importance
Looking at the feature importance scores from the random forest it is hard to know if we should keep the features or not. Moreover, if we wanted to consider feature importance from, say a non-linear SVM, we couldn't. Therefore we will now consider *permutation importance*.

<br>To do this we will use the [sklearn.inspection.permutation_importance](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html#sklearn.inspection.permutation_importance).

Recall we can either:
* compute our permutation importance based on a held-out test set (optionally repeating this multiple times to ensure we didn't hold out an "easy" or "hard" set by random chance).
* compute our permutation importance based on the full dataset after re-training our model on the full dataset (i.e. just before we might use it to predict) (not covered, ask and/or have a go if you're interested!)

In either case we're going to use `permutation_importance(...)` function from `sklearn.inspection`. The [documentation is here](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html#sklearn.inspection.permutation_importance).


The function takes three key arguments:
1. A fitted (trained) classifier/regressor
2. The input data to compute the permuation importance for
3. The output labels to compute the permuation importance for
4. Optional paramter: `scoring` - do you want accuracy or something else
4. The number of times to repeat the permuation process

The function returns a dictionary. Look at the documentation, but mostly you will be interested in the `.importance_means` which is a list of the variable importances (i.e. MDA) for the input variables, in the same order as in the input data.

### **Computing our permutation importance based on a held-out test set**

Basic steps:
1. Decide if you are going to select variables based on the variable importance scores or if you are just going to report the results. If you are just going to report the results you can use the test set we've previously made. Otherwise you need to further split the data into a sub_train and validation set as the variable selection should be considered like a "meta-paramter".
2. Train your model using either you training set or your sub_train set.
3. Use the `permutation_importance(...)` function to compute the permutation importance scores.

**TASK 2a.1:**
Implement a permutation importance based on a held-out test set. Assume we will ONLY REPORT the results.

Steps:
1. Using the fitted RandomForest model from the previous step use the `permutation_importance(...)` function to compute the permutation importance using the test set. Set n_repeates=3 to speed things up for this exersize.
2. In order to interpret the scores it is useful to know what the mean decrease in accuracy (that the feature importance scores represent) have decreased from. Compute the accuracy when using all features (i.e. predict and evaluate the test set).
3. Add the feature importances to the results dictionary and print the dictionary to compare the feature importances to those from the previous exercise. These can be accessed from the `.importances_mean` attribute in the returned object.

In [ ]:
from sklearn.inspection import permutation_importance

perm_imp_results = permutation_importance(rf, X_train, y_train, n_repeats=5)

In [ ]:


feature_importance_scores['RF_perm'] = perm_imp_results['importances_mean']

print_variable_importances( X_train.columns, feature_importance_scores, show_top = 10 )

Rank | Filter                                 | RF                                     | RF_perm                               
---- + -------------------------------------- + -------------------------------------- + --------------------------------------
0    | kw_max_avg                    : 0.0384 | kw_avg_avg                    : 0.0428 | kw_max_avg                    : 0.0155
1    | self_reference_avg_sharess    : 0.0340 | kw_max_avg                    : 0.0417 | kw_avg_avg                    : 0.0076
2    | LDA_03                        : 0.0296 | LDA_02                        : 0.0335 | self_reference_min_shares     : 0.0062
3    | self_reference_min_shares     : 0.0276 | self_reference_min_shares     : 0.0310 | is_weekend                    : 0.0056
4    | kw_avg_avg                    : 0.0251 | self_reference_avg_sharess    : 0.0310 | self_reference_avg_sharess    : 0.0026
5    | LDA_00                        : 0.0242 | LDA_04                        : 0.0307 | LDA_02         

**TASK 2a.2:**
Implement a permutation importance based on a held-out test set. Assume we will THEN DO FEATURE SELECTION.

Steps:
1. Further split our training set into a subtraining and validation (subtest) set. Use `test_size=0.33, random_state=42`.
2. Train a new RandomForest model (on X_train_sub, y_train_sub).
2. Using the fitted RandomForest model use the `permutation_importance(...)` function to compute the permutation importance using the validation (subtest) set.
2. In order to interpret the scores it is useful to know what the mean decrease in accuracy (that the feature importance scores represent) have decreased from. Compute the accuracy when using all features (i.e. predict and evaluate the test set).
3. Add the feature importances to the results dictionary and print the dictionary to compare the feature importances to those from the previous exercise.These can be accessed from the `.importances_mean` attribute in the returned object.

In [ ]:
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

rf_sub = RandomForestClassifier()
rf_sub.fit(X_train_sub, y_train_sub)

RandomForestClassifier()

In [ ]:
perm_imp_results_sub = permutation_importance(rf_sub, X_val, y_val, n_repeats=5)

# Compute accuracy when using all features on the validation (subtest) set
y_pred_val = rf_sub.predict(X_val)
accuracy_all_features = accuracy_score(y_val, y_pred_val)

# Add the feature importances to the results dictionary
feature_importance_scores['RF_perm_sub'] = perm_imp_results_sub['importances_mean']

# Print the dictionary to compare the feature importances to those from the previous exercise
print_variable_importances(X_train.columns, feature_importance_scores, show_top=10)

Rank | Filter                                 | RF                                     | RF_perm                                | RF_perm_sub                           
---- + -------------------------------------- + -------------------------------------- + -------------------------------------- + --------------------------------------
0    | kw_max_avg                    : 0.0384 | kw_avg_avg                    : 0.0428 | kw_max_avg                    : 0.0155 | kw_avg_avg                    : 0.0119
1    | self_reference_avg_sharess    : 0.0340 | kw_max_avg                    : 0.0417 | kw_avg_avg                    : 0.0076 | self_reference_min_shares     : 0.0086
2    | LDA_03                        : 0.0296 | LDA_02                        : 0.0335 | self_reference_min_shares     : 0.0062 | self_reference_avg_sharess    : 0.0080
3    | self_reference_min_shares     : 0.0276 | self_reference_min_shares     : 0.0310 | is_weekend                    : 0.0056 | is_weekend               

**TASK 2a.3:**
Implement a permutation importance based on a held-out test set. Assume we will THEN DO FEATURE SELECTION. Repeat the held-out split using a KFold cv stratergy. Use `sklearn.model_selection.StratifiedKFold` to do this. To keep things simple use k=5.

As a final step add the feature importances to the results dictionary and print the dictionary to compare the feature importances to those from the previous exercise.

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split

# Split the training data into subtraining and validation (subtest) sets
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

# Initialize a Random Forest Classifier
rf_sub = RandomForestClassifier()

# Train the Random Forest Classifier on the subtraining set
rf_sub.fit(X_train_sub, y_train_sub)

RandomForestClassifier()

In [ ]:
# Compute permutation importance on the validation (subtest) set
perm_imp_results_sub = permutation_importance(rf_sub, X_val, y_val, n_repeats=5, random_state=42)

In [ ]:
# Compute accuracy when using all features on the validation (subtest) set
y_pred_val = rf_sub.predict(X_val)
accuracy_all_features = accuracy_score(y_val, y_pred_val)

# Add the feature importances to the results dictionary
feature_importance_scores['RF_perm_sub'] = perm_imp_results_sub['importances_mean']

# Print the dictionary to compare the feature importances to those from the previous exercise
print_variable_importances(X_train.columns, feature_importance_scores, show_top=10)

Rank | Filter                                 | RF                                     | RF_perm                                | RF_perm_sub                           
---- + -------------------------------------- + -------------------------------------- + -------------------------------------- + --------------------------------------
0    | kw_max_avg                    : 0.0384 | kw_avg_avg                    : 0.0428 | kw_max_avg                    : 0.0155 | kw_avg_avg                    : 0.0166
1    | self_reference_avg_sharess    : 0.0340 | kw_max_avg                    : 0.0417 | kw_avg_avg                    : 0.0076 | is_weekend                    : 0.0097
2    | LDA_03                        : 0.0296 | LDA_02                        : 0.0335 | self_reference_min_shares     : 0.0062 | self_reference_avg_sharess    : 0.0064
3    | self_reference_min_shares     : 0.0276 | self_reference_min_shares     : 0.0310 | is_weekend                    : 0.0056 | self_reference_min_shares

### Part 2: Questions
Looking at the results what do you notice?

Specifically consider the permutation importance scores remembering their interpretation.

Try summing the features, what do you notice?

What can we infer from this?

In [ ]:
'''High consistency across different importance measures. Discrepancies between importance measures may suggest areas for further investigation.
For example, if a feature ranks high in RF but low in permutation importance, it might indicate that the feature is overfitting the model.
Features with consistently high importance scores across different methods are likely to be strong predictors and can be prioritized for further analysis or feature engineering.
Features with low consistency or inconsistent rankings may require additional scrutiny and possibly reevaluation of their relevance or representation.'''


## Part 3: Wrapper Methods
We know that considering the effect of holding out variables and/or their relative use within a given model does not show how *predictive* they actually are. This is because other variables may share information and other interaction effects may be going on.<br>

**Example:** If predicting someones country of birth the feature `country_of_drivers_licence` may not be listed as that predictive as it should have been if the feature `passport_country` exists since the model could have used either in a model dependent way. In random forests they are used equally meaning the predictive amount will be half of what it should have been on average in this case).

Wrapper methods perform feature subset search to try and find an optimal subset of features. Typically this is done for **feature selection** rather than **feature understanding**.

In the former we are interested in finding a fixed (or minimal) subset of features with the most predictive power. In the latter we are interested in asking questions regarding specific features (or groups of features) and how they individually and in combination with other features affect prediction accuracy. Here we are only going to consider the former. The latter currently must be done manually by holding out variables, writing your own permutation code or using R. See the lecture for more details.


### Part 3a: Recursive feature elimination (RFE)
Note that, in a lot of cases people will manually recursively eliminate features by running code similar to what we've seen in this practical and then remove features and try again. Why? As it (1) gives more control and (2) as with large datasets we may want to check things each iteration to prevent wasting compute time.

However, if we want to automate this, we can use an sklearn RFE Object.

In sklearn the RFE Object wraps a model. The model must produce `.feature_importances_` as part of the `.fit(...)` method or another attribute that contains the importances (specified via the `importance_getter` parameter).

Unfortunately, the `sklearn.inspection.permutation_importance` is a function that is separate to almost all models `.fit(..)` method. We therefore use a trick. We **change the definition of the .fit(..) method for the classifier/regressor of interest**.

The function below will do this for you, **resulting in the permutation importance scores being stored in `feature_importances_perm_`**.

The function modifies in-place an estimator **definition**. Use it like:
```
from sklearn.ensemble import RandomForestClassifier
add_or_remove_permutation_importance(RandomForestClassifier)
```

Then go on to use rf as the estimator in the sklearn RFE Object. **Do not forget to set the `feature_importances_perm_` parameter**, otherwise the model based importance, rather than permutation importance, scores will be used.

At the end, in addition to the attributes from the RFE Object

In [ ]:
def add_or_remove_permutation_importance( estimator, test_size = 0.33, random_state = 42, n_repeats = 5, save_performance_scores = True ):
    '''
    Modifies (or removes the modification if it exists) an sklearn estimator class .fit(..) to compute
    Permutation Importance based on a test set after fitting. The data passed to the fit function is
    first split into a training and test set, then the model trained and PI computed using the test set.

            Parameters:
                    estimator (sklearn estimator class): A class (not an object), such as RandomForestClassifier, RandomForestRegressor
                    test_size (float): The size of the (sub)test set size that will be used to computed the permutation importance
                    random_state (int): The random state used when the data is split
                    n_repeats (int): The number of repeats for the (sklearn) permutation importance function
                    save_performance_scores (bool): If True, computes the accuracy (MAE) for the classifier (regressor) on the held-out (sub)test set
                                                    and stores it in estimator.permutation_scores

            Returns:
                    None. The passed estimator class is modified in place.
    '''
    from sklearn.inspection import permutation_importance
    # define a new fit method that splits the data, trains the estimator
    # and then computes the
    def fit_with_perm(self, X,y,  sample_weight=None):
        # The extra permutation step - split the data
        X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X, y, test_size=test_size, random_state=random_state)
        # Now train the model
        self.fit_orig(X_train_sub,y_train_sub, sample_weight=sample_weight)
        # compute the feature importance on the test set and set the
        # expected attribute for the RFE Object
        self.feature_importances_perm_ = permutation_importance(self, X_test_sub, y_test_sub, n_repeats = n_repeats).importances_mean

        # If we asked to save the performance scores compute and record them
        if save_performance_scores:
            # Load a function to check if we have a classifier or regressor so we can use a correct evaluation measure
            from sklearn.base import is_classifier
            if is_classifier(estimator):
                from sklearn.metrics import accuracy_score
                estimator.permutation_scores[X.shape[1]] = accuracy_score(y_test_sub,self.predict(X_test_sub))
            else:
                from sklearn.metrics import mean_absolute_error
                estimator.permutation_scores[X.shape[1]] = accuracy_score(y_test_sub,self.predict(X_test_sub))

    # if we have previously added the permuatation fit instead
    # of the original fit, the estimator will have a fit_orig
    # attribute.
    if hasattr(estimator, 'fit_orig'):
        # if it does, put the .fit method back to the original one
        # and remove the dictionary we created to hold permutation scores if requested
        estimator.fit = estimator.fit_orig
        delattr(estimator, 'fit_orig')
        delattr(estimator, 'permutation_scores')
        print("Estimator's fit method returned to normal.")
    else:
        # store the actual fit method so we can (1) call it
        # in fit_with_perm(...) and (2) restore it later if we want
        estimator.fit_orig = estimator.fit
        # assign the estimator's .fit(..) to the new method that includes
        # permutation importance
        estimator.fit = fit_with_perm
        estimator.permutation_scores = {}
        print("Estimator's fit method set to include permutation importance.")





**TASKS:**
1. Create and fit RFE object (model) with and embedded Random Forest Classifier using Permutation Importance. Select 20 features. This will take about 12 minutes to run.
2. Print the generated feature importances for the selected features. This information is in the attribute `<rfe_object>.estimator_.permutation_scores`. <br>BONUS: Add a list of all features to the results dictionary with zeros when the feature wasn't selected. [Read the docs.](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html).
3. Print the generated feature importances for the selected features.
<br>BONUS: Add a list of all features to the results dictionary with zeros when the feature wasn't selected. [Read the docs.](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html).
4. For each model that was recusivly built with differing numbers of features (from the number of features up to 10) the score of the model was also computed on the held out (sub)test set. This provides an indication of how much predictive performance was lost at we reduced the number of features. View this information via: `<rfe_object>.estimator_.permutation_scores`.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

# Create a Random Forest Classifier
rf = RandomForestClassifier()

# Create RFE object with embedded Random Forest Classifier
rfe = RFE(estimator=rf, n_features_to_select=20)

# Fit the RFE object
rfe.fit(X_train, y_train)

# Get the names of all features
feature_names = X_train.columns

# Get the indices of the selected features
selected_feature_indices = np.where(rfe.support_)[0]

# Get the names of the selected features
selected_features = feature_names[selected_feature_indices]

# Print the names of the selected features
print("Selected Features:")
print(selected_features)

Selected Features:
Index(['n_tokens_content', 'n_unique_tokens', 'n_non_stop_unique_tokens',
       'average_token_length', 'kw_max_min', 'kw_avg_min', 'kw_avg_max',
       'kw_max_avg', 'kw_avg_avg', 'self_reference_min_shares',
       'self_reference_avg_sharess', 'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03',
       'LDA_04', 'global_subjectivity', 'global_sentiment_polarity',
       'global_rate_positive_words', 'avg_positive_polarity'],
      dtype='object')


In [ ]:
X_train_20 = X_train[selected_features]
# Compute permutation importance for all features
perm_imp_results = permutation_importance(rfe, X_train, y_train, n_repeats=5, random_state=42)

# Print the generated feature importances for all features
print("Feature Importances:")
print(perm_imp_results.importances_mean)

Feature Importances:
[0.         0.00137221 0.00201796 0.         0.00240137 0.
 0.         0.         0.         0.00167491 0.         0.
 0.         0.         0.         0.         0.         0.
 0.00205832 0.00347089 0.         0.         0.00760771 0.
 0.04471799 0.03953183 0.02786803 0.         0.02871557 0.
 0.         0.         0.         0.         0.         0.
 0.         0.0062355  0.01430734 0.00962567 0.00649783 0.01434769
 0.00417718 0.00458077 0.0017758  0.         0.         0.
 0.0008879  0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


In [ ]:
# Dictionary to store permutation scores for different numbers of features
permutation_scores = {}

# Iterate over different numbers of features
for n_features in range(1,11):  # From 1 to 10
    # Create RFE object with embedded Random Forest Classifier
    rfe = RFE(estimator=rf, n_features_to_select=n_features)

    # Fit the RFE object
    rfe.fit(X_train, y_train)

    # Compute permutation importance for the selected features on the test set
    perm_imp_results = permutation_importance(rfe, X_test, y_test, n_repeats=2, random_state=42)

    # Store the permutation scores for this model
    permutation_scores[n_features] = perm_imp_results.importances_mean



In [ ]:
# Print the permutation scores for different numbers of features
for n_features, scores in permutation_scores.items():
    print(f"Number of features: {n_features}, Permutation Scores: {scores}")

Number of features: 1, Permutation Scores: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.02441731 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
Number of features: 2, Permutation Scores: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.0487001  0.02448458 0.         0.         0.         0.
 0.         0.         0.         0.      

### Part 3b: All relevant feature selection
Instead of asking for a fixed number of features, we may simply want to throw away those that are not relevant. This is a more advanced method to **removing features with no/low variance**. I.e. we are looking to throw away features we think do not have any **real** impact. By real impact we mean no significant impact in a generalized sense. While we could have simply defined a threshold for RFE (stopping throwing away features once the permutation importance, i.e. change in mean prediction accuracy, was not zero) we would expect some change in prediction accuracy due to random chance. Addressing this issue the Boruta package finds iteratively throws aways the worst feature of the set, stopping when the change in mean prediction accuracy is greater than chance.

**Note: The approach is still a heuristic and may throw away valuable features since it removes features in a greedy fashion. In addition throwing away features now that look like they have no affect may be premature, as your training set grows they may be important.**

This is done by wrapping a model (again, one whose `.fit(..)` method produces `.feature_importances_`).

**TASKS:**
1. Fit a Boruta Object (model) by wrapping a RandomForestClassifier.
2. Print the selected features.

In [ ]:
# Remove the permuatation importance step from the fit - otherwise this will
# do a lot of compute for no reason and take a very long time!
add_or_remove_permutation_importance( RandomForestClassifier )

Estimator's fit method set to include permutation importance.


In [ ]:
!pip install -q Boruta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
X_train.head(2)

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
5028,7.0,2874.0,0.298141,1.0,0.466133,14.0,14.0,25.0,1.0,4.314892,...,0.398207,0.033333,1.0,-0.197937,-0.700000,-0.05,0.00,0.0,0.50,0.0
35501,12.0,224.0,0.660633,1.0,0.842105,3.0,1.0,1.0,1.0,4.629464,...,0.291667,0.100000,0.6,-0.191667,-0.333333,-0.05,0.05,0.0,0.45,0.0


In [ ]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier

# Assuming X_train and y_train are your training data
# Create a Random Forest Classifier
rf = RandomForestClassifier()

# Create Boruta object
boruta_selector = BorutaPy(estimator=rf, n_estimators='auto', verbose=2, random_state=42)

Check your predictors performance with all features vs just the features Boruta selected using your test set. Did it make it worse?

In [ ]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier

# Assuming X_train and y_train are your training data
# Create a Random Forest Classifier
rf = RandomForestClassifier()

rf.fit(X_train,y_train)

In [ ]:
np.int = np.int32
np.float = np.float64
np.bool = np.bool_

In [ ]:
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)
feat_selector.fit(X_train.values, y_train)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	58
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	58
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	58
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	58
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	58
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	58
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	58
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	10
Tentative: 	6
Rejected: 	42
Iteration: 	9 / 100
Confirmed: 	10
Tentative: 	6
Rejected: 	42
Iteration: 	10 / 100
Confirmed: 	10
Tentative: 	6
Rejected: 	42
Iteration: 	11 / 100
Confirmed: 	10
Tentative: 	6
Rejected: 	42
Iteration: 	12 / 100
Confirmed: 	10
Tentative: 	6
Rejected: 	42
Iteration: 	13 / 100
Confirmed: 	10
Tentative: 	5
Rejected: 	43
Iteration: 	14 / 100
Confirmed: 	10
Tentative: 	5
Rejected: 	43
Iteration: 	15 / 100
Confirmed: 	10
Tentative: 	5
Rejected: 	43
Iteration: 	16 / 100
Confirmed: 	11
Tentative: 	4
Reject

BorutaPy(estimator=RandomForestClassifier(n_estimators=50,
                                          random_state=RandomState(MT19937) at 0x7ADC1E8F5240),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7ADC1E8F5240, verbose=2)

In [ ]:
print( X_train.columns[feat_selector.support_] )

Index(['n_unique_tokens', 'n_non_stop_unique_tokens', 'kw_avg_min',
       'kw_avg_max', 'kw_max_avg', 'kw_avg_avg', 'self_reference_min_shares',
       'self_reference_avg_sharess', 'LDA_00', 'LDA_01', 'LDA_02', 'LDA_04',
       'global_subjectivity'],
      dtype='object')
